# OsuLearn
##### Machine Learning para jogar mapas de osu!

^^^

Isso aí, tô sem mais nada pra fazer, bora criar uma IA que joga osu! )o)

## Importar umas coisas...

In [1]:
# Machine Learning
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

# Plotagem
import matplotlib.pyplot as plt

# Utilidades
import os
import math

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Constantes

Aqui tem uns caminhos para os arquivos que precisamos:

In [2]:
# Pasta do osu!
OSU_FOLDER = "C:\\" + os.path.join("Program Files (x86)", "Jogos", "osu!")

# Nome do beatmap
BEATMAP = os.path.join('499478 Utsu-P - TRAUMATIC', 'Utsu-P - TRAUMATIC (Cryptic) [EXTREME].osu')
#BEATMAP = os.path.join('353412 Sakuzyo - Imprinting', 'Sakuzyo - Imprinting (Necroluttah) [Arles AR9.5].osu')

# Nome do arquivo de replay
REPLAY = 'BzMasked - Utsu-P - TRAUMATIC [EXTREME] (2018-12-17) Osu.osr'
#REPLAY = 'BzMasked - Sakuzyo - Imprinting [Arles AR9.5] (2018-12-17) Osu.osr'

## Leitura do arquivo de Beatmap

O formato do arquivo está especificado no [site do osu!](https://osu.ppy.sh/help/wiki/osu!_File_Formats/Osu_(file_format)/).

In [3]:
import osu.beatmap

beatmap_filename = os.path.join(OSU_FOLDER, "Songs", BEATMAP)
beatmap = osu.beatmap.load(beatmap_filename)

print()
print('Mapa:', beatmap['ArtistUnicode'], '-', beatmap['TitleUnicode'])
print('Autor:', beatmap['Creator'])
print('Dificuldade: ', beatmap['Version'])


Mapa: 鬱P - TRAUMATIC
Autor: Cryptic
Dificuldade:  EXTREME


## Leitura do arquivo de Replay

O formato do arquivo está especificado no [site do osu!](https://osu.ppy.sh/help/wiki/osu!_File_Formats/Osr_(file_format)/).

In [4]:
# replay = osu.replay.load(os.path.join(OSU_FOLDER, "Replays", REPLAY))

## Informações do Replay

Legal, carregamos o replay, isso foi o que achamos:

In [ ]:
# print()
# print("Player:", replay.player)

# print()
# print("Pontuação Total:", replay.score)
# print()
# print("%5d x 300\t%5d x 激" % (replay.n_300s, replay.n_geki))
# print("%5d x 100\t%5d x 喝" % (replay.n_100s, replay.n_katu))
# print("%5d x 50\t%5d x Erros" % (replay.n_50s, replay.n_misses))
# print()

# print("Acurácia: %.2f%%" % (100 * replay.accuracy))
# print("Combo Máximo: %dx" % replay.max_combo, "(Perfect)" if replay.perfect else "")
# print()

# %matplotlib inline

# u = [int(t[0]) / 1000 for t in replay.life_graph]
# v = [float(t[1]) for t in replay.life_graph]

# fig = plt.figure(facecolor='#333333')
# ax = fig.add_subplot(1, 1, 1)
# ax.set_xlim(0, max(u) + 1)
# ax.set_ylim(0, 1.1)
# ax.axis('off')
# ax.plot(u, v, '#ffcc22')
# plt.show()

## Visualização

Aqui temos uma previsualização do replay, também:

In [ ]:
%matplotlib notebook
#osu.replay.preview(beatmap, replay)

## Dados para treinamento...

In [ ]:
import osu.replay
import re

from glob import glob

replay_training_set = []

pattern = os.path.join(OSU_FOLDER, "Replays", "* Osu.osr")
for replay_file in glob(pattern):
    info = re.search(r"([^\\/]+) \- (.+ \- .+) \[(.+)\] \((.+)\)", replay_file)
    player, bmap, diff, date = info[1], info[2], info[3], info[4]
    print(player, bmap, diff, date)
    
    fpat = os.path.join(OSU_FOLDER, "Songs", "*" + bmap + "*", "*[[]" + diff + "[]].osu")
    
    for beatmap_file in glob(fpat):
        beatmap = osu.beatmap.load(beatmap_file)
        if re.search("v14", beatmap.format_version) != None:
            replay = osu.replay.load(replay_file)
            if len(replay.data) < 10000:
                replay_training_set.append([beatmap, replay])
                print("OK")
        else:
            print("Wrong format version")
            
    print()
    

BzMasked Aimer with chelly (EGOIST) - ninelie (REDSHiFT x Vesuvia remix) Daydream 2018-06-01

BzMasked Aitsuki Nakuru - Monochrome Butterfly Extra 2017-09-09
OK

BzMasked Aitsuki Nakuru - Monochrome Butterfly Mysterious 2017-08-04

BzMasked Alex Gaudino feat. Crystal Waters - Destination Calabria Destination Unknown 2017-11-25

BzMasked Alex Gaudino feat. Crystal Waters - Destination Calabria Destination Unknown 2017-12-01

BzMasked Ata - Euphoria Juggernaut 285bpm Tap jumps 2017-09-24
OK

BzMasked Ayase Rie - Yuima-ruWorld TVver. Ultimate 2017-08-14

BzMasked Camellia - Exit This Earth's Atomosphere Evolution 2017-08-13

BzMasked Camellia - Exit This Earth's Atomosphere Evolution 2018-09-27

BzMasked Caravan Palace - Lone Digger Funky 2018-09-28

BzMasked Coalamode. - Nanairo Symphony -TV Size- Kaori's Free Spirit 2017-11-19

BzMasked Coalamode. - Nanairo Symphony -TV Size- Kaori's Free Spirit 2017-11-20

BzMasked DECO27 - Ghost Rule z1085684963's Extra 2018-10-07

BzMasked DJ TOTTO f

## Finalmente: Modelo de rede neural

Agora começa a festa...

In [ ]:
from osu.learn.OsuLearnModel import create_training_data, create_target_data
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, CuDNNLSTM, concatenate, Input, Reshape, Conv1D, UpSampling1D

REPLAY_FEATURES = ['x', 'y']
MAP_FEATURES = ['x', 'y', 'time_left']

X = create_training_data(replay_training_set)
y = create_target_data(replay_training_set)

print(X.shape, y.shape)

(16, 9955, 3) (16, 9955, 2)


In [ ]:
DROPOUT_RATE = 0.2

model = Sequential()

map_input = Input(shape=(X.shape[1], X.shape[2]), name='map_info')

#replay_input = Input(shape=(y.shape[1], y.shape[2]), name='replay_info')

#dense = Dense(y.shape[1] * len(REPLAY_FEATURES), activation='linear')(dense)

#out = Dense(64, activation='linear')(concatenate([lstm, dense]))
#out = Reshape([len(MAP_FEATURES) * len(REPLAY_FEATURES)])(out)

out = CuDNNLSTM(len(MAP_FEATURES), return_sequences=True)(map_input)
out = Conv1D(256, X.shape[1], activation='linear')(out)

out = Dense(X.shape[1] * len(REPLAY_FEATURES), activation='linear')(out)
out = Dropout(DROPOUT_RATE)(out)

out = Reshape([X.shape[1], len(REPLAY_FEATURES)])(out)

model = Model(inputs=map_input, outputs=out)
# model.add(Dropout(DROPOUT_RATE))

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(DROPOUT_RATE))

# model.add(Dense(512, activation='relu'))
# model.add(Dropout(DROPOUT_RATE))

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(DROPOUT_RATE))

# model.add(Dense(128, activation='relu'))
# model.add(Dropout(DROPOUT_RATE))

#model.add(Dense(len(REPLAY_FEATURES), activation='relu'))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
map_info (InputLayer)        (None, 9955, 3)           0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 9955, 3)           96        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 256)            7645696   
_________________________________________________________________
dense_1 (Dense)              (None, 1, 19910)          5116870   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 19910)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 9955, 2)           0         
Total params: 12,762,662
Trainable params: 12,762,662
Non-trainable params: 0
________________________________________________________________

In [ ]:
def plot_info(compare):
    %matplotlib inline
    
    for data in compare:
        plt.plot([(x) for x, y in data])
    plt.show()
    
    for data in compare:
        plt.plot([(y) for x, y in data])
    plt.show()
    
#     for data in compare:
#         plt.plot([(a) for distance, angle in data])
#     plt.show()
    
#     for data in compare:
#         plt.plot([(b) for distance, angle in data])
    plt.show()

In [ ]:
history = model.fit(X, y, batch_size=64, epochs=32)

Epoch 1/128
16/16 [==============================] - 6s 352ms/step - loss: 0.2247 - acc: 0.5055
Epoch 2/128
16/16 [==============================] - 1s 65ms/step - loss: 0.2178 - acc: 0.5152
Epoch 3/128
16/16 [==============================] - 1s 65ms/step - loss: 0.1908 - acc: 0.5172
Epoch 4/128
16/16 [==============================] - 1s 65ms/step - loss: 0.1460 - acc: 0.5285
Epoch 5/128
16/16 [==============================] - 1s 67ms/step - loss: 0.1634 - acc: 0.5429
Epoch 6/128
16/16 [==============================] - 1s 65ms/step - loss: 0.1428 - acc: 0.5506
Epoch 7/128
16/16 [==============================] - 1s 64ms/step - loss: 0.1210 - acc: 0.5557
Epoch 8/128
16/16 [==============================] - 1s 64ms/step - loss: 0.1298 - acc: 0.5592
Epoch 9/128
16/16 [==============================] - 1s 64ms/step - loss: 0.1243 - acc: 0.5658
Epoch 10/128
16/16 [==============================] - 1s 63ms/step - loss: 0.1134 - acc: 0.5691
Epoch 11/128
16/16 [============================

In [ ]:
%matplotlib inline

plt.plot(history.history['acc'])
plt.show()

plt.plot(history.history['loss'])
plt.show()

In [ ]:
predicted = model.predict(X)

for i in range(len(y)):
    plot_info([y[i], predicted[i]])

In [ ]:
%matplotlib notebook

replay_data = []

j = 0
for i in range(len(predicted[0])):
    if len(replay_training_set[0][1].data) > i + j:
        w = replay_training_set[0][1].data[i + j][0]
        while w < 0 and len(replay_training_set[0][1].data) > i + j + 1:
            j += 1
            w = replay_training_set[0][1].data[i + j][0]
    else:
        w = 0
    cx, cy = predicted[0][i]
    replay_data.append([w, cx * 512, cy * 384, 0])
    
#osu.replay.preview(replay_training_set[0][0], replay_data)

## 